In [ ]:
!pip install ions

In [ ]:
import io
from ase.io import read

# create string of a .cif file
cif = """
# generated using pymatgen
data_Li3PO4
_symmetry_space_group_name_H-M   'P 1'
_cell_length_a   4.89264480
_cell_length_b   6.07079561
_cell_length_c   10.41629663
_cell_angle_alpha   90.00000000
_cell_angle_beta   90.00000000
_cell_angle_gamma   90.00000000
_symmetry_Int_Tables_number   1
_chemical_formula_structural   Li3PO4
_chemical_formula_sum   'Li12 P4 O16'
_cell_volume   309.38741088
_cell_formula_units_Z   4
loop_
 _symmetry_equiv_pos_site_id
 _symmetry_equiv_pos_as_xyz
  1  'x, y, z'
loop_
 _atom_type_symbol
 _atom_type_oxidation_number
  Li+  1.0
  P5+  5.0
  O2-  -2.0
loop_
 _atom_site_type_symbol
 _atom_site_label
 _atom_site_symmetry_multiplicity
 _atom_site_fract_x
 _atom_site_fract_y
 _atom_site_fract_z
 _atom_site_occupancy
  Li+  Li0  1  0.69534509  0.49846062  0.83568861  1
  Li+  Li1  1  0.19534509  0.50153938  0.66431139  1
  Li+  Li2  1  0.30465491  0.99846062  0.16431139  1
  Li+  Li3  1  0.80465491  0.00153938  0.33568861  1
  Li+  Li4  1  0.30465491  0.50153938  0.16431139  1
  Li+  Li5  1  0.80465491  0.49846062  0.33568861  1
  Li+  Li6  1  0.69534509  0.00153938  0.83568861  1
  Li+  Li7  1  0.19534509  0.99846062  0.66431139  1
  Li+  Li8  1  0.79721728  0.25000000  0.57624687  1
  Li+  Li9  1  0.29721728  0.75000000  0.92375313  1
  Li+  Li10  1  0.20278272  0.75000000  0.42375313  1
  Li+  Li11  1  0.70278272  0.25000000  0.07624687  1
  P5+  P12  1  0.69067243  0.75000000  0.58814555  1
  P5+  P13  1  0.80932757  0.75000000  0.08814555  1
  P5+  P14  1  0.30932757  0.25000000  0.41185445  1
  P5+  P15  1  0.19067243  0.25000000  0.91185445  1
  O2-  O16  1  0.62473411  0.25000000  0.41081835  1
  O2-  O17  1  0.79434655  0.95731182  0.65858046  1
  O2-  O18  1  0.12473411  0.75000000  0.08918165  1
  O2-  O19  1  0.37526589  0.75000000  0.58918165  1
  O2-  O20  1  0.79434655  0.54268818  0.65858046  1
  O2-  O21  1  0.29434655  0.04268818  0.84141954  1
  O2-  O22  1  0.20565345  0.45731182  0.34141954  1
  O2-  O23  1  0.70565345  0.54268818  0.15858046  1
  O2-  O24  1  0.20565345  0.04268818  0.34141954  1
  O2-  O25  1  0.70565345  0.95731182  0.15858046  1
  O2-  O26  1  0.79484118  0.75000000  0.44883378  1
  O2-  O27  1  0.29434655  0.45731182  0.84141954  1
  O2-  O28  1  0.70515882  0.75000000  0.94883378  1
  O2-  O29  1  0.20515882  0.25000000  0.55116622  1
  O2-  O30  1  0.29484118  0.25000000  0.05116622  1
  O2-  O31  1  0.87526589  0.25000000  0.91081835  1
"""

# read structure
atoms = read(io.StringIO(cif), format = 'cif')
atoms

Atoms(symbols='Li12P4O16', pbc=True, cell=[4.8926448, 6.07079561, 10.41629663], spacegroup_kinds=...)

In [50]:
from ions import Decorator # or you can use pymatgen's BVAnalyzer (which is faster)

# decorate atoms with oxi states
atoms = Decorator().decorate(atoms)

In [ ]:
from ions.featurizers import StructureFeaturizer

# define featurizer
specie = 3 # Li
featurizer = StructureFeaturizer(atoms, specie, oxi=True)

# get features to predict Li-ion percolation barrier
fea = featurizer.featurize(stats = ['mean', 'max']) # 

In [99]:
from ions.featurizers import EdgeFeaturizer
from ions.geom import Edge

In [105]:
source, target, offset = 1, 8, [-1, 0, 0] # Li-Li jump from source to target
# define edge 
edge = Edge(atoms, source, target, offset)

# define featurizer
featurizer = EdgeFeaturizer(edge, oxi = True)

# get features to predict Li-ion migration barrier
fea = featurizer.featurize(stats = ['mean', 'min', 'std'])

In [107]:
import pandas as pd
from ions.tools import Percolator

pl = Percolator(atoms, 3, 8.0)
tr = 0.5 # min allowed distance between the edge and nearest neighbors

# find minimum jump distance cutoff corresponding to the maximum percolation dimnesionality
cutoff, dim = pl.mincut_maxdim(tr)

#  find unique edges forming a percolating network
edges, _ = pl.unique_edges(cutoff = cutoff, tr = tr)

# collect features for each edge
mp_id = 'mp-2878'
features = []
for edge in edges:
    fea = EdgeFeaturizer(edge, oxi = True).featurize()
    edge_id = f'{mp_id}_{edge.source}_{edge.target}_{edge.offset[0]}_{edge.offset[1]}_{edge.offset[2]}'
    fea.update({'mp_id': mp_id})
    fea.update({'edge_id': edge_id})
    features.append(fea)

data = pd.DataFrame(features)
data

,length,min_dist_to_edge,cylinder_volume,cylinder_volume^1/3,cylinder_ionic_volume,cylinder_ionic_volume^1/3,min_CN,max_CN,mean_CN,range_CN,...,has_O,has_F,has_P,has_S,has_B,has_I,has_Cl,has_N,mp_id,edge_id
0,3.016707,1.177850,8.573030,2.857677,2.143258,0.714419,6.275524,7.494469,6.812681,1.218946,...,1,0,1,0,0,0,0,0,mp-2878,mp-2878_0_6_0_0_0
1,3.028445,1.232314,14.448149,4.816050,3.612037,1.204012,6.275524,7.518656,6.886097,1.243132,...,1,0,1,0,0,0,0,0,mp-2878,mp-2878_0_1_0_0_0
2,2.924913,1.238204,14.087941,4.695980,3.521985,1.173995,6.275524,6.905136,6.573718,0.629612,...,1,0,1,0,0,0,0,0,mp-2878,mp-2878_0_11_0_0_1
3,2.639625,1.463474,17.244432,5.748144,4.311108,1.437036,6.275524,6.902079,6.546270,0.626555,...,1,0,1,0,0,0,0,0,mp-2878,mp-2878_0_9_0_0_0
